# 데이콘 Basic Summer
## 서울 랜드마크 이미지 분류 경진대회


# I. 데이터 살펴보기

## 1. 데이터 준비
### 1.1 csv 데이터

In [3]:
# pandas 사용하여 데이터 불러오기

import pandas as pd
label_df = pd.read_csv('dataset/train.csv')
label_df.head()

,file_name,label
0,001.PNG,9
1,002.PNG,4
2,003.PNG,1
3,004.PNG,1
4,005.PNG,6


### 1.2 이미지 데이터

데이터 이미지의 local address와 label 값을 list에 저장

In [4]:
import os
from glob import glob
# CNN 모델에서 꼭 필요한 두가지 모듈
# 'glob' : 파일의 경로명을 리스트로 뽑을 때 사용

def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir,'*.PNG')))
    img_path_list = list(map(lambda x: x.replace('\\','/', 10), img_path_list))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))

    # get label
    label_list.extend(label_df['label'])
    
    #print(img_path_list)
    #print(label_list)
    
    return img_path_list,label_list

def get_test_data(data_dir):
    img_path_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir,'*.PNG')))
    img_path_list = list(map(lambda x: x.replace('\\','/', 10), img_path_list))
    img_path_list.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))
    #print(img_path_list)
    
    return img_path_list

## 2. 데이터 확인
### 2.1. csv 데이터

pandas의 `info()` 메소드 활용하여 데이터의 특성 살펴보기

In [5]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  723 non-null    object
 1   label      723 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.4+ KB


### 2.2. 이미지 데이터

In [6]:
all_img_path, all_label = get_train_data('dataset/train')
test_img_path = get_test_data('dataset/test')

#['dataset/train/001.PNG',
#['dataset/train\\001.PNG',

In [7]:
all_label[:5]

[9, 4, 1, 1, 6]

In [8]:
all_img_path[:5]

['dataset/train/001.PNG',
 'dataset/train/002.PNG',
 'dataset/train/003.PNG',
 'dataset/train/004.PNG',
 'dataset/train/005.PNG']

In [9]:
test_img_path[:5]

['dataset/test/001.PNG',
 'dataset/test/002.PNG',
 'dataset/test/003.PNG',
 'dataset/test/004.PNG',
 'dataset/test/005.PNG']

## 3. 환경설정
데이터를 전처리 하기위함 GPU 딥러닝 환경설정

In [10]:
import torch
import torch.nn as nn

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVISE_ORDER"] = "PCI_BUS_ID"  # rrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # Set the GPU 2 to use, 멀티 GPU

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [11]:
torch.cuda.is_available()

False

In [12]:
# GPU 체크 및 할당

if torch.cuda.is_available():
    #device = torch.device("cuda:0")
    print("Device:",device)
    print("There are %d GPU(s) available"%torch.cuda.device_count())
    print("We will use the GPU:",torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU availavle, using the CPU instead.")

No GPU availavle, using the CPU instead.


# 똥컴이라 안됨

In [13]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

1.12.1
False


In [14]:
# 하이퍼 파라미터 튜닝
CFG = {
    'IMG_SIZE':128, # 이미지 사이즈
    'EPOCHS':50, # 에포크 - 전체 데이터를 사용하여 학습하는 횟수
    'LEARNING_RATE':2e-2, # 학습률
    'BATCH_SIZE':12, #배치사이즈
    'SEED':41, #시드
}

In [15]:
# 모델의 재현성을 위해 random seed 고정

import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #torch.bakcends.cudnn.deterministic = True
    #torch.backends cudnn benchmark = True
    
seed_everything(CFG['SEED'])

# II. 데이터 전처리
## 2. CustomDataset
CustomDataset을 만들어 전체 dataset을 구성

In [16]:
import torchvision.datasets as datasets  # 이미지 데이터셋 집합체
import torchvision.transforms as transforms  # 이미지 변환 툴 - 텐서 변환, 이미지 정규화...

from torch.utils.data import DataLoader  # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

import cv2   # 이미지 출력하고, 저장하고, 새 윈도우에 보여주고 등등

class CustomDataset(Dataset):
    #필요한 변수 선언
    def __init__(self,img_path_list,label_list,train_mode = True, transforms = None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list
    
    # index번째 data를 return
    def __getitem__(self,index):
        img_path = self.img_path_list[index]  # 이미지 파일 경로
        print(img_path)
        
        # Get image data
        image = cv2.imread(img_path)  # 이미지 파일
        if self.transforms is not None:   # transfrom 형태가 주어지면 transfrom하여 저장
            image = self.transforms(image)
            
        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):  # 길이 출력
        return len(self.img_path_list)
            

In [17]:
# 임시 데이터셋 만들어 정상 작동하는지 확인
tempdataset = CustomDataset(all_img_path, all_label,train_mode = False)

In [18]:
import matplotlib.pyplot as plt

#plt.imshow(tempdataset.__getitem__(0))

# Kernal Died ?? Prolly GPU 사용 못해서 메모리 사용량 과다로 죽은듯

## 3. Train / Validation Split

학습시킬 데이터셋과 검증할 데이터셋 분리!

In [19]:
# Train : Validation = 0.75: 0.25 Split

train_len = int(len(all_img_path)*0.75)
Vali_len = int(len(all_img_path)*0.25)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

In [20]:
print("train set 길이:", train_len)
print("validation set 길이:", Vali_len)

train set 길이: 542
validation set 길이: 180


## 4. Transfrom

나뉜 데이터셋에서 이미지를 분석하기 위해 이미지 변형(transform) 적용

In [21]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),  # Numpy 배열에서 PIL 이미지로 변형
    transforms.Resize([CFG['IMG_SIZE'],CFG['IMG_SIZE']]), # 이미지 사이즈 변형
    transforms.ToTensor(), #이미지 데이터를 tensor
    transforms.Normalize(mean= (0.5,0.5,0.5),std = (0.5,0.5,0.5)) # 이미지 정규화
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]),\
    transforms.ToTensor(),
    transforms.Normalize(mean =(0.5,0.5,0.5),std = (0.5,0.5,0.5))
])

## 5. Dataloader
- `Dataloader` Class 생성: batch 기반 딥러닝모델 학습을 위해 mini batch를 만들어주는 역할

- dataset의 전체 데이터가 batch size로 나뉨

- 만들었던 dataset을 input으로 넣어주면 여러 옵션(데이터 묶기, 섞기, 알아서 병렬처리)를 통해 batch 생성

In [22]:
# Get Dataloader

# CustomDataset class를 통하여 train dataset 생성
train_dataset = CustomDataset(train_img_path, 
                              train_label, 
                              train_mode = True, 
                              transforms = train_transform)
# 만든 train dataset를 DataLoader에 넣어 batch 만들기
train_loader = DataLoader(train_dataset,
                          batch_size = CFG['BATCH_SIZE'],
                          shuffle=True, 
                          num_workers =0)


# Validation 에서도 적용
vali_dataset = CustomDataset(vali_img_path, 
                             vali_label, 
                             train_mode = True, 
                             transforms = test_transform)
vali_loader = DataLoader(vali_dataset,
                         batch_size = CFG['BATCH_SIZE'],
                         shuffle=True, 
                         num_workers =0)

In [23]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('total train imgs: ',train_len,'/ total train batches(iterations) :',train_batches)
print('total valid imgs: ',Vali_len,'/ total valid batches(iterations) :',vali_batches)

total train imgs:  542 / total train batches(iterations) : 46
total valid imgs:  180 / total valid batches(iterations) : 16


In [24]:
print(vali_loader)

**train 데이터의 경우, 542 장의 이미지를, 12개짜리 배치로 나누어 1번의 epoch를 돌기 위해 46개의 batch를 반복 학습해야한다.**

- 배치 사이즈 : 12 
- train - 46 묶음 / validation - 16 묶음

# III. 모델링
## 1. 모델 구조 정의
CNN모델을 이해하기 위한 기본 개념

### - Neural Network (신경망)
신경망은 다른 말로 인공 신경망이라고도 불린다.

신경망이란 뇌 속 뉴런의 망형 구조를 닮은 다층형 구조의 컴퓨팅 모델이다.
여기에서 서로 연결된 처리 소자, 일명 '뉴런'이라는 것이 있으며 생물학적 뉴런을 수학적으로 모델린한 것인데, 이들이 서로 협력하여 출력 함수를 도출한다.

신경망은 입력 및 출력 계층/차원으로 구성되며 대부분은 숨겨진 계층으로 이루어져 있다. 숨겨진 계층은 입력을 출력 계층에서 사용할 수 있는 무언가로 변환해주는 단위로 구성된다. 따라서, 인공신경망 뉴런은 여러 입력값을 받아 일정 수준을 넘어서게 되면 활성화되고 출력값을 내보낸다.

### 1) Activation Function (활성화 함수)

딥러닝 네트워크에서 노드에 입력된 값들을 비선형 함수에 통과시킨 후 다음 레이어로 전달하는 데, 이 떄 사용하는 함수를 활성화 함수(Activation Function)이라 한다. 

선형 함수가 아니라 비선형 함수를 사용하는 이유는 딥러닝 모델의 레이어 층을 깊게 가져갈 수 있기 때문이다.

**대표적인 활성화 함수 살펴보기**

#### 1.1) Sigmoid 함수

Sigmoid 함수는 Logistic 함수라고 불리기도 하며, x의 값에 따라 0~1의 값을 출력하는 S자형 함수이다.

`Binary Classification` 마지막 레이어의 활성화 함수로 사용된다.

Sigmoid 함수는 Activation function으로 많이 사용되며, 보통 0.5 미만은 0, 이상은 1을 출력한다.

$$sigmoid(x)\\
= 1 / (1 + exp(-x))\\
= exp(x) / (exp(x) + 1)$$

![시그모이드함수](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/480px-Logistic-curve.svg.png)

그래프를 보면, 양 극단 값의 기울기(미분 값)이 0에 가까워져, 학습이 되지 않는 문제가 발생한다.

이를 **Gradient Vanishing 문제**라고 하는데 이러한 문제를 해결하기 위해 다양한 활성화함수들이 고안되었다.

### 2) Perceptron (퍼셉트론)

퍼셉트론이란 신경망의 기원이 되는 개념으로, Frank Rosenblatt가 1957년에 고안한 알고리즘이다. 

이는 여러 신호를 입력받아 일정한 임계값을 넘는지를 확인하여, 0(흐르지 않는다, 임계값을 넘지 않았다) 또는 1(흐른다, 임계값을 넘었다)라는 출력값을 앞으로 전달한다.

**(뉴런의 all-or-none 늒김)**

각 입력 신호 X는 각 가중치 w와 곱해진다. 가중치가 클수록 그 신호가 중요하다는 뜻이다.

그러나, 퍼셉트론은 단순한 선형 분류기로, AND나 OR과 같은 분류는 가능하나, XOR 분류는 불가능하다.

![퍼셉트론 러닝 알고리즘](https://image.slidesharecdn.com/machine-learning-120930145310-phpapp01/95/machine-learning-with-applications-in-categorization-popularity-and-sequence-labeling-75-638.jpg?cb=1354541953)


**편향(bias) = $theta$ (임계치)** 

: 노드를 얼마나 쉽게 활성화(1로 출력; activation) 되느냐를 조정하는 매개변수
* theta (임계치)가 높아질수록, 엄격한 모델이 되며, underfitting의 문제 발생 가능
* theat (임계치)가 낮아질수록, 느슨한 모델이 되며, overfitting, 과도한 noise 포함 등의 문제 발생 


[<링크> 개념에 대한 자세한 설명](https://velog.io/@skyepodium/%ED%8D%BC%EC%85%89%ED%8A%B8%EB%A1%A0%EC%9D%B4-%EC%99%9C-XOR-%EB%AC%B8%EC%A0%9C%EB%A5%BC-%EB%AA%BB%ED%91%B8%EB%8A%94%EC%A7%80-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0)

### 3) MLP (Multi-Layer Perceptron 다층 퍼셉트론)

직선형(선형) 영역만 구분하고 표시할 수 있는 단층 퍼셉트론의 한계를 극복하기 위해 고안된 것이 다층 퍼셉트론이다.

XOR 문제의 경우, 각 영역을 직선으로 분리하는게 불가능한데, 비선형 영역까지 표현 가능한 다층 퍼셉트론으로 XOR 게이트까지 구현할 수 있다.

![다층 퍼셉트론 구조](https://velog.velcdn.com/images%2Fskyepodium%2Fpost%2F10424d38-94ea-4228-aeda-37757f7bfa99%2F%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-05-19%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%201.11.02.png)

![다층 퍼셉트론 XOR](https://velog.velcdn.com/images%2Fskyepodium%2Fpost%2F63930671-cb82-43a2-825b-e822beec415f%2F%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-05-19%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%201.03.37.png)


--------

In [25]:
from tqdm.auto import tqdm
import torch.nn as nn # 신경망들이 포함됨
#import torch.nn.init as init # 텐서에 초기값을 줌

class CNNclassification(torch.nn.Module):
    def __init__(self):
        super(CNNclassification, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=2, stride=2)) #pooling layer
        
        self.fc_layer = nn.Sequential( 
            nn.Linear(3136, 10) #fully connected layer(ouput layer)
        )    
        
    def forward(self, x):
        
        x = self.layer1(x) #1층
        
        x = self.layer2(x) #2층
         
        x = self.layer3(x) #3층
        
        x = self.layer4(x) #4층
        
        x = torch.flatten(x, start_dim=1) # N차원 배열 -> 1차원 배열
        
        out = self.fc_layer(x)
        return out

## Convolution Layer

Convolution Layer에서는 이미지의 특징(feature map)을 추출해낸다.

입력 데이터가 주어지면 필터를 이용해 특징을 추출한 다음 아웃풋을 내보낸다.

이 필터는 `커널(kernel)` 혹은 `가중치의 배열`이라고도 부르며, 이 값을 조정하는 것이 곧 학습을 의미한다.

![convolution layer](https://cdn-images-1.medium.com/max/1200/1*1okwhewf5KCtIPaFib4XaA.gif)

**Blue: Input / Green: Output**

**2D convolution using a kernel size of 3, stride of 1 and padding**


![convolution input_channel output_channel example](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbptGfh%2FbtrxiFKPKnF%2FvxsCM7XDcvFHBoWbWFKpQK%2Fimg.png)


### Kernel size

Convolution의 시야(view)를 결정한다. 보통 2D에서 3x3 pixel로 사용한다.

첫번째 Convolution Layer에서도 3x3 크기의 커널을 사용했다. 이는 곧 학습해야할 가중치가 9개라는 뜻이다. 


### Stride

이미지를 횡단할 때 커널의 스텝사이즈를 결정한다. 즉, 커널을 이동시키는 거리이다.

기본값을 1이지만, 보통 max pooling과 비슷하게 이미지를 다운샘플링하기 위해 stride를 2로 사용하기도 한다.

Covolution을 하게되면 입력 데이터의 크기가 자연스럽게 줄어들게 되는데, 주변에 값이 0인 셀들을 추가(Padding)해서 입력 데이터의 크기를 유지시키기도 한다.


### Padding

샘플의 테두리를 어떻게 조절할 지를 결정한다. 

Zero padding은 이미지 주위에 0을 둘러서, layer를 통과할 때 이미지 데이터의 축소를 방지해주는 역할을 한다. 

패딩된 convolution은 input과 동일한 output 차원을 유지하는 반면, 패딩되지 않은 convolution은 커널이 1보다 큰 경우 테두리의 일부를 잘라버릴 수 있다. 


### ReLU

활성화 함수 중 하나이다. 

Gradient vanishing 문제를 해결 가능하고 계산이 빠르고 양 극단값이 포화되지 않는다는 장점이 있다.


### Pooling Layer ⊃ Max Pooling

Pooling layer는 데이터의 공간적 크기를 축소하는 데 사용한다.

보통 이 레이어에서 이미지의 크기를 조절하며, CNN에서는 주로 Max-pooling 방식을 사용한다. 

Conv layer은 이미지의 특정 영역의 특징을 잡아내는 역할이라면, Pooling은 이미지의 크기를 줄이는 동시에 이미지 전체를 볼 수 있게 도와준다.

Max pooling에서는 선택된 영역에서 가장 큰 값을 뽑아 대표값으로 설정한다.

이를 통해 **학습 시간을 단축**하고 **오버피팅 문제**를 완화할 수 있다.


### Fully Connected (output) Layer

이전 레이어의 출력을 평탄화(flatten)하여 다음 스테이지의 입력이 될 수 있는 **단일 벡터로 변환**한다.

마지막으로 각 라벨에 대한 **최종 확률**을 제공한다.


### Input & Output channels

Convolution layer는 input 채널의 특정 수($I$)를 받아 output 채널의 특정 수($O$)를 계산한다. 

K를 커널의 수라고 할 때, 이런 계층에서 필요한 파라미터의 수는 $I \times O \times K$로 계산할 수 있다. 


![CNN 전과정](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FDSJmz%2FbtrxiXFh1M2%2FK1LX9HCz4dEskKOK9WT4Vk%2Fimg.png)

![CNN 전과정 2](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbasjDA%2Fbtrxr497kPE%2FxjyADtaFK3LPI6TAqBdaK1%2Fimg.jpg)

## 2. 모델 학습

모델 학습을 하기위해 매개변수 정의하기

In [26]:
import torch.optim as optim  #최적화 알고리즘들을 포함하는 패키지

model = CNNclassification().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(),lr = CFG['LEARNING_RATE'])
scheduler = None

### Classification model

이번 베이스라인 프로젝트 모델에서는 기본 **`CNN classification 모델`** 사용했다.

### Loss function

손실함수로는 classification 문제이기 때문에 **`CrossEntropyLoss`**를 사용했다.

손실함수는 실제 값과 모델이 예측한 값의 거리를 출력하는 함수로, 예측이 얼마나 틀렸는지 알려주는 함수이다.

"모델의 예측이 얼마나 틀렸는지"를 어떻게 정의하느냐에 따라 어떤 Loss function을 사용할 지가 정해진다. 


### Optimizer

최적화 함수로는 확률적 경사 하강법 **`SGD(Stochastic Gradient Descent)`**를 사용했다.

Optimizer는 학습 데이터(Train data)를 이용하여 모델을 학습할 때 데이터의 실제 결과와 오델이 예측한 결과를 기반으로 오차를 잘 줄일 수 있게 만들어주는 역할을 한다.

여기서 Learning rate, 학습률은 얼마나 빠른 속도로 이동하는 지를 결정한다.

Learning rate를 엄청 크게 설정한다면 원하는 값까지 빠르게 도달할 수 있지만, 자칫하면 오히려 최소값에 수렴하지 못할 수도 있다.

반면 너무 작은 경우에는 최소값에 도달할 수 있을 지는 몰라도, 시간이 매우 오래 걸리고, overfitting의 문제가 발생할 수 있다.

따라서 적절한 learning rate를 설정하는 과정이 중요하다.

이제 train 함수를 만들고 train 데이터를 학습시켜 validation으로 평가하는 메소드를 작성해보자.

In [27]:
from tqdm import tqdm  # 반복문의 어디까지 반복이 진행되었는지 표시!!

def train(model, optimizer, train_loader, scheduler, device):
    model.to(device)
    n = len(train_loader)
    best_acc = 0  # 0으로 초기화
    
    for epoch in range(1, CFG["EPOCHS"]+1):  # 에포크 설정
        model.train()  #모델 학습
        running_loss = 0.0  # 0으로 초기화
        
        for img,label in tqdm(iter(train_loader)):
            img,label = img.to(device),label.to(device)  # 배치 데이터
            optimizer.zero_grad()  # 배치마다 optimizer 초기화
            
            # Data -> Model -> Output
            logit = model(img)  # 예측값 산출, logit : 최종값, 스코어
            loss = criterion(logit, label)  # `criterion()` : 손실함수 계산 (앞에서 crossentropy로 정함)
            
            # Back Propagation 역전파
            loss.backward()  # 손실함수 기준 역전파
            optimizer.step()  # 가중치 업데이트
            running_loss += loss.item()
            
            print(running_loss)
        
        print('[%d] Train loss: %.10f' %(epoch, running_loss/len(train_loader)))
        
        if scheduler is not None:
            scheduler.step()   # 하이퍼파라미터 업데이트
            
            
            
        # ***************Validation set 평가****************
        
        model.eval()  # evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시킴
        vali_loss = 0.0
        correct = 0
        with torch.no_grad():  # 파라미터 업데이트 안하기 때문에 no_grad 사용  
            for img,label in tqdm(iter(vali_loader)):
                img,label = img.to(device), label.to(device)
                
                logit = model(img)
                vali_loss += criterion(logit,label)
                pred = logit.argmax(dim = 1, keepdim =True) # 11개의 class 중 가장 값이 높은 것을 예측 label로 추출
                correct += pred.eq(label.view_as(pred)).sum().item()  # 에측값과 실제값이 맞으면 1, 아니면 0
        vali_acc = 100 * correct / len(vali_loader.dataset)
        print('[Vali set] Avg Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader),correct, len(vali_loader.dataset), 100 * correct/ len(vali_loader.dataset)))

        
        # 베스트 모델 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            torch.save(model.state_dict(),'best_model.pth') # 이 디렉토리에 best_model.pth를 저장
            print('Model Saved.')

In [28]:
#for img,label in  tqdm(iter(train_loader)):
    #print("************",i,"************")
    #print("img:",img)
    #print("label:",label)

### Logit 

확률화되지 않은 예측 결과를 logit이라고 한다. 확률값을 계산하지 않고 layer를 거쳐 나온 산출물을 그대로 다음 layer에 넘길 때 사용한다.

즉, logit은 신경망의 최종 레이어가 내놀은 확률 값이 아닌 중간 결과물이고, 결과값의 범위가 실수 전체이다. 

![logit](https://velog.velcdn.com/images%2Fguide333%2Fpost%2Feb996ddf-1ff4-48e7-b8f9-fa23b622505a%2FScreenshot%20from%202021-05-17%2011-01-37.png)

Classification 문제이기 때문에 평가지표로는 Accuracy를 사용하여 모델의 정확도를 산출하였다.


### Epoch

딥러닝에서 epoch는 전체 트레이닝 셋이 신경망을 통과한 횟수이다.

1-epoch는 전체 트레이닝 셋이 하나의 신경망에 적용되어 순전파와 역전파를 통해 한번 통과했다는 뜻이다.

Epoch가 많다고 꼭 학습이 잘 되는 것은 아니다. 
Epoch가 너무 적을 경우 학습이 덜 이루어질 수 있고, 너무 많을 경우 과적합이 되는 경우도 있다. 따라서, 적절한 epoch를 설정하는 게 중요하다.

이때 validation loss와 accuracy는 epoch를 언제 중단 할 지 모니터링하는 용도로 사용되기도 한다.


### Batch size

Batch size란 CPU 또는 GPU 연산 시, 하드웨어로 한번에 로드되는 데이터의 개수이다. 

개인 컴퓨터 환경(메모리 용량)에 따라 batch size를 조절해야하며, 이는 모델 학습 과정에 영향을 끼치기도 하기 때문에, 적절히 설정하는 것이 중요하다.


### Backpropagation (역전파)

오차 역전파법이라고도 하며 예측값과 실제값의 차이인 오차를 계산하고고, 이것을 다시 역으로 input 단에 보내, 오차가 작아지는 방향으로 가중치를 수정하는 과정을 일정 횟수 반복하는 방법이다.

이때 역전파 과정에서는 앞서 언급했던 `최적화 함수`를 이용한다.

In [29]:
train(model, optimizer, train_loader, scheduler, device)

  0%|                                                                                           | 0/46 [00:00<?, ?it/s]

dataset/train/204.PNG
dataset/train/435.PNG
dataset/train/341.PNG
dataset/train/436.PNG
dataset/train/121.PNG
dataset/train/398.PNG
dataset/train/170.PNG
dataset/train/375.PNG
dataset/train/106.PNG
dataset/train/130.PNG
dataset/train/213.PNG
dataset/train/187.PNG


  2%|█▊                                                                                 | 1/46 [00:00<00:26,  1.67it/s]

2.305204153060913
dataset/train/450.PNG
dataset/train/039.PNG
dataset/train/522.PNG
dataset/train/528.PNG
dataset/train/048.PNG
dataset/train/465.PNG
dataset/train/166.PNG
dataset/train/088.PNG


  4%|███▌                                                                               | 2/46 [00:01<00:22,  1.99it/s]

...

 24%|███████████████████▌                                                              | 11/46 [00:05<00:16,  2.09it/s]

dataset/train/441.PNG
dataset/train/403.PNG
dataset/train/302.PNG
dataset/train/009.PNG
25.33716917037964
dataset/train/078.PNG
dataset/train/104.PNG


 24%|███████████████████▌                                                              | 11/46 [00:05<00:18,  1.91it/s]

dataset/train/457.PNG
dataset/train/259.PNG
dataset/train/420.PNG
dataset/train/012.PNG
dataset/train/408.PNG


KeyboardInterrupt: 

In [ ]:
best_acc

[Vali set] Avg Loss: 0.9572, Accuracy: 140/181 ( 77%)

Model Saved.

**Epoch가 [   ]일 때 Validation Accuracy가 [  ]%로 best_model에 선정되어 저장되었다.**

## 3. 추론하기 (Inference) - Test 데이터에 적용하기

이제 학습된 best_model을 가지고 test 셋의 라벨을 추론해보도록 하자.

`with torch.no_grad():`

: Autograd 엔진을 꺼버려 더이상 자동으로 gradient 트래킹을 하지 않는다. 이렇게 함으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함이다. 
 
 <br/>

`model.eval()`

: train 데이터를 활용하는 모델링 단계와 test 데이터로 추론하는 단계에서 다르게 동작하는 layer들이 존재한다. 예를 들면, Dropout layer나 BatchNorm 같은 레이어는 학습 시에만 동작한다. 

이런 layer들의 동작을 inference(evaluation) mode로 바꿔주는 목적으로 사용된다. 

**따라서, inference를 진행할 때에는 두가지 메소드를 다 호출해야한다!**

In [30]:
def predict(model,test_loader,device):
    model.eval()  # evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시킴
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.to(device)
            
            pred_logit = model(img)
            pred_logit = pred_logit.argmax(dim =1, keepdim = True).squeeze(1)
            
            model_pred.extend(pred_logit.tolist())
    return model_pred

In [31]:
test_dataset = CustomDataset(test_img_path,
                             None,
                             train_mode = False,
                             transforms= test_transform
                            )
test_loader = DataLoader(test_dataset,
                        batch_size = CFG['BATCH_SIZE'],
                         shuffle = False,
                         num_workers = 0
                        )


# Validation Accuracy가 가장 뛰|어난 모델을 불러온다
checkpoint = torch.load('best_model.pth')
model = CNNclassification().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)
preds[0:5]

  0%|                                                                                           | 0/17 [00:00<?, ?it/s]

dataset/test/001.PNG
dataset/test/002.PNG
dataset/test/003.PNG
dataset/test/004.PNG
dataset/test/005.PNG
dataset/test/006.PNG
dataset/test/007.PNG
dataset/test/008.PNG
dataset/test/009.PNG


  6%|████▉                                                                              | 1/17 [00:00<00:05,  3.15it/s]

dataset/test/010.PNG
dataset/test/011.PNG
dataset/test/012.PNG
dataset/test/013.PNG
dataset/test/014.PNG
dataset/test/015.PNG
dataset/test/016.PNG
dataset/test/017.PNG
dataset/test/018.PNG
dataset/test/019.PNG
dataset/test/020.PNG
dataset/test/021.PNG
dataset/test/022.PNG
dataset/test/023.PNG
dataset/test/024.PNG


 12%|█████████▊                                                                         | 2/17 [00:00<00:05,  2.95it/s]

...

dataset/test/129.PNG
dataset/test/130.PNG
dataset/test/131.PNG
dataset/test/132.PNG
dataset/test/133.PNG
dataset/test/134.PNG
dataset/test/135.PNG
dataset/test/136.PNG
dataset/test/137.PNG
dataset/test/138.PNG
dataset/test/139.PNG
dataset/test/140.PNG


 71%|█████████████████████████████████████████████████████████▉                        | 12/17 [00:04<00:01,  2.54it/s]

dataset/test/141.PNG
dataset/test/142.PNG
dataset/test/143.PNG
dataset/test/144.PNG
dataset/test/145.PNG
dataset/test/146.PNG
dataset/test/147.PNG
dataset/test/148.PNG
dataset/test/149.PNG
dataset/test/150.PNG
dataset/test/151.PNG
dataset/test/152.PNG
dataset/test/153.PNG
dataset/test/154.PNG
dataset/test/155.PNG
dataset/test/156.PNG


 71%|█████████████████████████████████████████████████████████▉                        | 12/17 [00:04<00:02,  2.49it/s]


KeyboardInterrupt: 

# IV. 데이콘 제출하기

## 1. Submission 파일 생성

이제 예측한 값을 sample_submission.csv에 넣어 제출용 파일을 생성하자.
제출한 뒤 리더보드를 통해 결과 확인.

In [32]:
submission  = pd.read_csv('dataset/sample_submission.csv')
submission['label'] = preds

NameError: name 'preds' is not defined

In [80]:
submission.head()

,file_name,label
0,001.PNG,7
1,002.PNG,1
2,003.PNG,9
3,004.PNG,7
4,005.PNG,6


In [81]:
submission.to_csv('submission/submission1.csv',index=False)  
#index False 꼭 넣기

In [33]:
submission = pd.read_csv('submission/submission1.csv')

In [44]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  199 non-null    object
 1   label      199 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [46]:
samplesubmission = pd.read_csv('dataset/sample_submission.csv')

In [47]:
samplesubmission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  199 non-null    object
 1   label      199 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB
